In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import Callback
from sklearn.preprocessing import MinMaxScaler
from google.colab import files

# Upload file CSV
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Baca data
data = pd.read_csv(filename)
print("Data awal:")
print(data.head())

# Normalisasi data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.values)

# Persiapan data
look_back = 0
X = np.reshape(scaled_data, (scaled_data.shape[0], 1, scaled_data.shape[1]))
y = scaled_data[:, -1]

# Custom callback untuk mencari prediksi yang meningkat
class IncreasingPredictionCallback(Callback):
    def __init__(self, X_data, threshold):
        super().__init__()
        self.X_data = X_data
        self.threshold = threshold  # Nilai minimal yang harus dilewati
        self.found_prediction = None
        self.best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        if self.found_prediction is None:  # Jika belum ketemu yang memenuhi syarat
            current_pred = self.model.predict(self.X_data, verbose=0)[0][0]
            if current_pred > self.threshold:
                self.found_prediction = current_pred
                self.best_weights = self.model.get_weights()
                self.model.stop_training = True  # Stop training begitu ketemu

# Buat model LSTM
def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(32, input_shape=input_shape))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# List untuk menyimpan hasil
final_predictions = []
previous_prediction = -np.inf  # Inisialisasi dengan nilai sangat kecil

# Training per baris (percobaan)
for i in range(len(X)):
    print(f"\n=== Proses Training Percobaan {i+1} ===")
    # print(f"Mencari prediksi > {previous_prediction:.6f}")

    # Inisialisasi model
    model = create_model((X.shape[1], X.shape[2]))

    # Data untuk percobaan ini
    X_exp = X[i:i+1]
    y_exp = y[i:i+1]

    # Buat callback dengan threshold = previous_prediction
    increasing_cb = IncreasingPredictionCallback(X_exp, previous_prediction)

    try:
        # Training model sampai ketemu prediksi yang lebih besar
        history = model.fit(X_exp, y_exp,
                          epochs=100,
                          batch_size=1,
                          verbose=0,
                          callbacks=[increasing_cb])

        if increasing_cb.found_prediction is not None:
            current_pred = increasing_cb.found_prediction
            # print(f"Prediksi ditemukan: {current_pred:.6f}")
            print(f"Percobaan selesai")

            # Update model dengan weights terbaik
            model.set_weights(increasing_cb.best_weights)

            # Simpan hasil
            final_predictions.append(current_pred)
            previous_prediction = current_pred
        else:
            print(f"Percobaan selesai")
            final_predictions.append(previous_prediction)

    except Exception as e:
        print(f"Error: {str(e)}")
        final_predictions.append(float('nan'))
        continue

# Proses hasil prediksi
valid_predictions = [p for p in final_predictions if not np.isnan(p)]

# Denormalize predictions
predictions = np.array(valid_predictions).reshape(-1, 1)
dummy = np.zeros((len(predictions), scaled_data.shape[1]))
dummy[:, -1] = predictions.flatten()
denorm_predictions = scaler.inverse_transform(dummy)[:, -1]

# Plot hasil
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(denorm_predictions)+1), denorm_predictions, 'ro-')
plt.title('Percobaan Nilai Kualitas Minyak')
plt.xlabel('Percobaan Ke-')
plt.ylabel('Nilai Percobaan')
plt.grid(True)

# Tampilkan hasil dalam tabel
results = pd.DataFrame({
    'Percobaan': range(1, len(denorm_predictions)+1),
    'Nilai Percobaan': denorm_predictions,
    'Kenaikan': np.diff(np.insert(denorm_predictions, 0, np.nan))  # Hitung selisih
})
print("\nHasil Percobaan:")
print(results)

print("\nUrutan Percobaan:")
for i, pred in enumerate(denorm_predictions, 1):
    print(f"Percobaan {i}: {pred:.6f}")